## Stepwise Backward Elimination (Wrapper Method)

**Stepwise Backward Elimination** is a _wrapper-based_, _greedy_ feature selection algorithm.  
It starts with **all available features** and removes them **one at a time** based on their impact on model performance.

---

### Key Idea

Begin with the full model and iteratively **remove the least useful feature** at each step.

- The algorithm is **sequential**, not exhaustive.
- It evaluates the effect of **removing one feature at a time**.
- Once a feature is removed, it **cannot be added back**.

---

### 🔹 Algorithm Steps

1. **Start with all features**  
   Fit the model using the complete feature set.

2. **Evaluate feature removal**  
   For each feature:

- Temporarily remove that feature
- Fit the model with the remaining features
- Evaluate performance (e.g., RMSE, accuracy, ROC AUC, cross-validation score)

3. **Remove the worst feature**  
   Permanently remove the feature whose removal:

- Improves performance the most, or
- Degrades performance the least

4. **Repeat**  
   Continue removing features until:

- A desired number of features is reached, or
- Performance drops beyond a threshold, or
- No further improvement is observed

---

### Example

Suppose you have features: **A, B, C, D**

**Step 0:** Start with `{A, B, C, D}`

**Step 1:** Try removing each feature:

- Remove A → `{B, C, D}` → score = 0.82
- Remove B → `{A, C, D}` → score = 0.78
- Remove C → `{A, B, D}` → score = 0.81
- Remove D → `{A, B, C}` → score = 0.84 ✅

➡ Remove **D** permanently

**Step 2:** Current set `{A, B, C}`  
Try removing one more:

- Remove A → `{B, C}` → score = 0.80
- Remove B → `{A, C}` → score = 0.75
- Remove C → `{A, B}` → score = 0.83 ✅

➡ Remove **C**

**Final selected features:** `{A, B}`

---

### Important Characteristics

- Does **not** evaluate all feature combinations
- Computationally cheaper than exhaustive search
- Can get stuck in a **local optimum**
- Early removal mistakes cannot be corrected

---

### Comparison with Forward Selection

| Aspect        | Forward Selection            | Backward Elimination             |
| ------------- | ---------------------------- | -------------------------------- |
| Start         | No features                  | All features                     |
| Action        | Add one feature              | Remove one feature               |
| Suitable when | Few useful features          | Many useful features             |
| Cost          | Lower when features are many | Expensive when features are many |

---

### When to Use Backward Elimination

- When the number of features is **moderate**
- When you believe **most features are useful**
- When model interpretability is important
- When computational cost is acceptable

---


### Classification


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.datasets import load_iris, load_diabetes

In [27]:
X = pd.DataFrame(load_iris().data, columns=load_iris().feature_names)
y = pd.Series(load_iris().target)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [28]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

selector = SequentialFeatureSelector(
    estimator=rf,
    n_features_to_select='auto',   # You can specify an integer or 'auto'
    tol=None,                      # Tolerance for stopping criterion
    direction='backward',          # 'forward' or 'backward'
    scoring='accuracy',            # default is accuracy for classifiers. You can specify others like 'f1', 'roc_auc', etc.
    cv=3,
    n_jobs=4)

In [29]:
selector.fit(X_train, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:813: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:830: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:830: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:830: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_tar

,estimator estimator: estimator instanceAn unfitted estimator.,RandomForestC...ndom_state=42)
,"n_features_to_select n_features_to_select: ""auto"", int or float, default=""auto""If `""auto""`, the behaviour depends on the `tol` parameter:- if `tol` is not `None`, then features are selected while the score change does not exceed `tol`.- otherwise, half of the features are selected.If integer, the parameter is the absolute number of features to select.If float between 0 and 1, it is the fraction of features to select... versionadded:: 1.1 The option `""auto""` was added in version 1.1... versionchanged:: 1.3 The default changed from `""warn""` to `""auto""` in 1.3.",'auto'
,"tol tol: float, default=NoneIf the score is not incremented by at least `tol` between twoconsecutive feature additions or removals, stop adding or removing.`tol` can be negative when removing features using `direction=""backward""`.`tol` is required to be strictly positive when doing forward selection.It can be useful to reduce the number of features at the cost of a smalldecrease in the score.`tol` is enabled only when `n_features_to_select` is `""auto""`... versionadded:: 1.1",None
,"direction direction: {'forward', 'backward'}, default='forward'Whether to perform forward selection or backward selection.",'backward'
,"scoring scoring: str or callable, default=NoneScoring method to use for cross-validation. Options:- str: see :ref:`scoring_string_names` for options.- callable: a scorer callable object (e.g., function) with signature ``scorer(estimator, X, y)`` that returns a single value. See :ref:`scoring_callable` for details.- `None`: the `estimator`'s :ref:`default evaluation criterion ` is used.",'accuracy'
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used. In all othercases, :class:`~sklearn.model_selection.KFold` is used. These splittersare instantiated with `shuffle=False` so the splits will be the sameacross calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.",3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel. When evaluating a new feature toadd or remove, the cross-validation procedure is parallel over thefolds.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",4
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2


In [30]:
selector.get_feature_names_out()

array(['s2', 's3', 's4', 's5', 's6'], dtype=object)

### Regression


In [31]:
X = pd.DataFrame(load_diabetes().data, columns=load_diabetes().feature_names)
y = pd.Series(load_diabetes().target)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [33]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)

selector = SequentialFeatureSelector(
    estimator=rf,
    n_features_to_select='auto',   # You can specify an integer or 'auto'
    tol=None,                      # Tolerance for stopping criterion
    direction='backward',          # 'forward' or 'backward'
    scoring='r2',            # default is accuracy for classifiers. You can specify others like 'f1', 'roc_auc', etc.
    cv=3,
    n_jobs=4)

In [34]:
selector.fit(X_train, y_train)

,estimator estimator: estimator instanceAn unfitted estimator.,RandomForestR...ndom_state=42)
,"n_features_to_select n_features_to_select: ""auto"", int or float, default=""auto""If `""auto""`, the behaviour depends on the `tol` parameter:- if `tol` is not `None`, then features are selected while the score change does not exceed `tol`.- otherwise, half of the features are selected.If integer, the parameter is the absolute number of features to select.If float between 0 and 1, it is the fraction of features to select... versionadded:: 1.1 The option `""auto""` was added in version 1.1... versionchanged:: 1.3 The default changed from `""warn""` to `""auto""` in 1.3.",'auto'
,"tol tol: float, default=NoneIf the score is not incremented by at least `tol` between twoconsecutive feature additions or removals, stop adding or removing.`tol` can be negative when removing features using `direction=""backward""`.`tol` is required to be strictly positive when doing forward selection.It can be useful to reduce the number of features at the cost of a smalldecrease in the score.`tol` is enabled only when `n_features_to_select` is `""auto""`... versionadded:: 1.1",None
,"direction direction: {'forward', 'backward'}, default='forward'Whether to perform forward selection or backward selection.",'backward'
,"scoring scoring: str or callable, default=NoneScoring method to use for cross-validation. Options:- str: see :ref:`scoring_string_names` for options.- callable: a scorer callable object (e.g., function) with signature ``scorer(estimator, X, y)`` that returns a single value. See :ref:`scoring_callable` for details.- `None`: the `estimator`'s :ref:`default evaluation criterion ` is used.",'r2'
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used. In all othercases, :class:`~sklearn.model_selection.KFold` is used. These splittersare instantiated with `shuffle=False` so the splits will be the sameacross calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.",3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel. When evaluating a new feature toadd or remove, the cross-validation procedure is parallel over thefolds.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",4
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmi

In [37]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

rf.fit(X_train_selected, y_train)
y_pred_selected = rf.predict(X_test_selected)

rf.fit(X_train, y_train)
y_pred_full = rf.predict(X_test)

r2_selected = r2_score(y_test, y_pred_selected)
r2_full = r2_score(y_test, y_pred_full)

print(f"R2 Score with selected features: {r2_selected:.4f}")
print(f"R2 Score with all features: {r2_full:.4f}")

R2 Score with selected features: 0.4426
R2 Score with all features: 0.4703


The difference between the R² scores obtained using all features (R² ≈ 0.47) and using the selected features (R² ≈ 0.44) is negligible. This indicates that, after feature selection, the predictive performance of the model remains largely unchanged. In other words, we were able to reduce the input dimensionality to only two features while sacrificing only a very small amount of predictive accuracy. This reduction leads to simpler models, improved interpretability, and lower computational cost during both training and inference.

More importantly, the absolute value of the R² score should be interpreted relatively rather than in isolation. An R² in the range of 0.4–0.5 does not necessarily indicate weak performance. In many real-world regression problems, a substantial portion of the variance in the target variable is driven by unobserved factors, measurement noise, or inherently stochastic processes. As a result, even well-performing models may explain only a moderate fraction of the total variance.
